In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.types import * 
import pyspark.sql.functions as F
from pyspark.sql.functions import col, asc,desc
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from pyspark.sql import SQLContext
from pyspark.mllib.stat import Statistics
import pandas as pd
from pyspark.sql.functions import udf
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler,StandardScaler
from pyspark.ml import Pipeline
from sklearn.metrics import confusion_matrix

spark=SparkSession.builder \
.master ("local[*]")\
.appName("week3")\
.getOrCreate()
sc=spark.sparkContext
sqlContext=SQLContext(sc)

22/10/23 00:01:12 WARN Utils: Your hostname, Ramcharans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.4.30 instead (on interface en0)
22/10/23 00:01:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/23 00:01:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/23 00:01:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/23 00:01:14 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/23 00:01:14 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/10/23 00:01:14 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


/usr/local/opt/apache-spark/libexec/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
#Different data formats - CSV, TXT, JSON, PARQUET

dataframe_json = spark.read.json('/usr/local/opt/apache-spark/libexec/examples/src/main/resources/people.json') 
dataframe_txt = spark.read.text('/usr/local/opt/apache-spark/libexec/examples/src/main/resources/people.txt') 
dataframe_csv = spark.read.csv('/usr/local/opt/apache-spark/libexec/examples/src/main/resources/people.csv') 
dataframe_parquet = spark.read.load('/usr/local/opt/apache-spark/libexec/examples/src/main/resources/users.parquet')

In [4]:
dataframe_json.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [5]:
dataframe_txt.show()

+-----------+
|      value|
+-----------+
|Michael, 29|
|   Andy, 30|
| Justin, 19|
+-----------+



In [6]:
dataframe_csv.show()

+------------------+
|               _c0|
+------------------+
|      name;age;job|
|Jorge;30;Developer|
|  Bob;32;Developer|
+------------------+



In [7]:
dataframe_parquet.show()

+------+--------------+----------------+
|  name|favorite_color|favorite_numbers|
+------+--------------+----------------+
|Alyssa|          null|  [3, 9, 15, 20]|
|   Ben|           red|              []|
+------+--------------+----------------+



In [8]:
from pyspark.sql.types import Row
list_rdd=sc.parallelize([('Messi',35),('Ronaldo',37),('Salah',30),('Neymar',30),('Mbappe',23)])

players = list_rdd.map(lambda x: Row(name=x[0], age=int(x[1])))
players_df = spark.createDataFrame(players)
players_df.show()

+-------+---+
|   name|age|
+-------+---+
|  Messi| 35|
|Ronaldo| 37|
|  Salah| 30|
| Neymar| 30|
| Mbappe| 23|
+-------+---+



In [9]:
pwd

'/Users/ramcharankakarla/BAN5753/Week3'

In [10]:
#load the data file

df= spark.read.option('header','true').option('inferschema','true')\
.csv('/Users/ramcharankakarla/BAN5753/Week3/employees.csv')


In [11]:
#display the records
df.show(5)

+----------+------+----------+---------------+------+-------+-----------------+---------------+
|First Name|Gender|Start Date|Last Login Time|Salary|Bonus %|Senior Management|           Team|
+----------+------+----------+---------------+------+-------+-----------------+---------------+
|   Douglas|  Male|  8/6/1993|       12:42 PM| 97308|  6.945|             true|      Marketing|
|    Thomas|  Male| 3/31/1996|        6:53 AM| 61933|   4.17|             true|           null|
|     Maria|Female| 4/23/1993|       11:17 AM|130590| 11.858|            false|        Finance|
|     Jerry|  Male|  3/4/2005|        1:00 PM|138705|   9.34|             true|        Finance|
|     Larry|  Male| 1/24/1998|        4:47 PM|101004|  1.389|             true|Client Services|
+----------+------+----------+---------------+------+-------+-----------------+---------------+
only showing top 5 rows



In [12]:
df.printSchema()


root
 |-- First Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Start Date: string (nullable = true)
 |-- Last Login Time: string (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Bonus %: double (nullable = true)
 |-- Senior Management: boolean (nullable = true)
 |-- Team: string (nullable = true)



# count()
The count() function counts the number of rows in DataFrame.

In [13]:
df.count()

1000

# columns()
The columns() function returns the columns of DataFrame.

In [14]:
df.columns

['First Name',
 'Gender',
 'Start Date',
 'Last Login Time',
 'Salary',
 'Bonus %',
 'Senior Management',
 'Team']

# describe()
The describe(n) function computes the summary statistics of selected (n) columns in DataFrame.

In [15]:
df.describe("Salary","Bonus %").show()

+-------+------------------+-----------------+
|summary|            Salary|          Bonus %|
+-------+------------------+-----------------+
|  count|              1000|             1000|
|   mean|         90662.181|        10.207555|
| stddev|32923.693342022256|5.528480814753191|
|    min|             35013|            1.015|
|    max|            149908|           19.944|
+-------+------------------+-----------------+



# select()
The select() function shows only selected column or columns of DataFrame.

In [16]:
df.select("First Name","Gender").show(5)

+----------+------+
|First Name|Gender|
+----------+------+
|   Douglas|  Male|
|    Thomas|  Male|
|     Maria|Female|
|     Jerry|  Male|
|     Larry|  Male|
+----------+------+
only showing top 5 rows



In [17]:
df.select("First Name","Gender").show()

+----------+------+
|First Name|Gender|
+----------+------+
|   Douglas|  Male|
|    Thomas|  Male|
|     Maria|Female|
|     Jerry|  Male|
|     Larry|  Male|
|    Dennis|  Male|
|      Ruby|Female|
|      null|Female|
|    Angela|Female|
|   Frances|Female|
|    Louise|Female|
|     Julie|Female|
|   Brandon|  Male|
|      Gary|  Male|
|  Kimberly|Female|
|   Lillian|Female|
|    Jeremy|  Male|
|     Shawn|  Male|
|     Diana|Female|
|     Donna|Female|
+----------+------+
only showing top 20 rows




# when()
The when() function shows selected column and designated cases depending on condition 

In [18]:
from pyspark.sql.functions import when
df.select("First Name",when(df.Salary > 10000, 1).otherwise(0)).show(5)

+----------+--------------------------------------------+
|First Name|CASE WHEN (Salary > 10000) THEN 1 ELSE 0 END|
+----------+--------------------------------------------+
|   Douglas|                                           1|
|    Thomas|                                           1|
|     Maria|                                           1|
|     Jerry|                                           1|
|     Larry|                                           1|
+----------+--------------------------------------------+
only showing top 5 rows



# like()
The like() function shows selected column and filter out the condition we are looking for by assigning “True” or “False”.

In [19]:
df.select("First Name", df['Team'].like("%Client%")).show(5)

+----------+------------------+
|First Name|Team LIKE %Client%|
+----------+------------------+
|   Douglas|             false|
|    Thomas|              null|
|     Maria|             false|
|     Jerry|             false|
|     Larry|              true|
+----------+------------------+
only showing top 5 rows



# Startswith()
The startswish() function searchs the letters in the brackets starting from the beginning of the first word in specified column.

In [20]:
df.select("First Name",df.Team.startswith("Mar")).show(5)

+----------+---------------------+
|First Name|startswith(Team, Mar)|
+----------+---------------------+
|   Douglas|                 true|
|    Thomas|                 null|
|     Maria|                false|
|     Jerry|                false|
|     Larry|                false|
+----------+---------------------+
only showing top 5 rows



# Endswith()
The endswith() function searchs the letters in the brackets starting from the end of the last word in specified column.

In [21]:
df.select("First Name",df.Team.endswith("ing")).show(5)

+----------+-------------------+
|First Name|endswith(Team, ing)|
+----------+-------------------+
|   Douglas|               true|
|    Thomas|               null|
|     Maria|              false|
|     Jerry|              false|
|     Larry|              false|
+----------+-------------------+
only showing top 5 rows



# Substring()
The substring() function extracts the substring from the column with provided length of the string you wanted to extract.

In [22]:
df.select(df["Gender"].substr(1, 1).alias("Gender_Ind")).show(5)

+----------+
|Gender_Ind|
+----------+
|         M|
|         M|
|         F|
|         M|
|         M|
+----------+
only showing top 5 rows



# between()
The between() function filters column between provided conditions.

In [23]:
df.select("First Name",df.Salary.between(1000, 10000)).show(5)

+----------+----------------------------------------+
|First Name|((Salary >= 1000) AND (Salary <= 10000))|
+----------+----------------------------------------+
|   Douglas|                                   false|
|    Thomas|                                   false|
|     Maria|                                   false|
|     Jerry|                                   false|
|     Larry|                                   false|
+----------+----------------------------------------+
only showing top 5 rows



# withColumn()
The withcolumn() function adds or replaces columns in an existing DataFrame. 

In [24]:
df.withColumn("yearly_salary",df.Salary*12).show(5)


+----------+------+----------+---------------+------+-------+-----------------+---------------+-------------+
|First Name|Gender|Start Date|Last Login Time|Salary|Bonus %|Senior Management|           Team|yearly_salary|
+----------+------+----------+---------------+------+-------+-----------------+---------------+-------------+
|   Douglas|  Male|  8/6/1993|       12:42 PM| 97308|  6.945|             true|      Marketing|      1167696|
|    Thomas|  Male| 3/31/1996|        6:53 AM| 61933|   4.17|             true|           null|       743196|
|     Maria|Female| 4/23/1993|       11:17 AM|130590| 11.858|            false|        Finance|      1567080|
|     Jerry|  Male|  3/4/2005|        1:00 PM|138705|   9.34|             true|        Finance|      1664460|
|     Larry|  Male| 1/24/1998|        4:47 PM|101004|  1.389|             true|Client Services|      1212048|
+----------+------+----------+---------------+------+-------+-----------------+---------------+-------------+
only showi

In [25]:
df.show()

+----------+------+----------+---------------+------+-------+-----------------+--------------------+
|First Name|Gender|Start Date|Last Login Time|Salary|Bonus %|Senior Management|                Team|
+----------+------+----------+---------------+------+-------+-----------------+--------------------+
|   Douglas|  Male|  8/6/1993|       12:42 PM| 97308|  6.945|             true|           Marketing|
|    Thomas|  Male| 3/31/1996|        6:53 AM| 61933|   4.17|             true|                null|
|     Maria|Female| 4/23/1993|       11:17 AM|130590| 11.858|            false|             Finance|
|     Jerry|  Male|  3/4/2005|        1:00 PM|138705|   9.34|             true|             Finance|
|     Larry|  Male| 1/24/1998|        4:47 PM|101004|  1.389|             true|     Client Services|
|    Dennis|  Male| 4/18/1987|        1:35 AM|115163| 10.125|            false|               Legal|
|      Ruby|Female| 8/17/1987|        4:20 PM| 65476| 10.012|             true|            

In [26]:
df=df.withColumn("yearly_salary",df.Salary*12)

In [30]:
df.show()

+----------+------+----------+---------------+------+-------+-----------------+--------------------+-------------+
|First Name|Gender|Start Date|Last Login Time|Salary|Bonus %|Senior Management|                Team|yearly_salary|
+----------+------+----------+---------------+------+-------+-----------------+--------------------+-------------+
|   Douglas|  Male|  8/6/1993|       12:42 PM| 97308|  6.945|             true|           Marketing|      1167696|
|    Thomas|  Male| 3/31/1996|        6:53 AM| 61933|   4.17|             true|                null|       743196|
|     Maria|Female| 4/23/1993|       11:17 AM|130590| 11.858|            false|             Finance|      1567080|
|     Jerry|  Male|  3/4/2005|        1:00 PM|138705|   9.34|             true|             Finance|      1664460|
|     Larry|  Male| 1/24/1998|        4:47 PM|101004|  1.389|             true|     Client Services|      1212048|
|    Dennis|  Male| 4/18/1987|        1:35 AM|115163| 10.125|            false| 

# Cast()

In [31]:
df.withColumn("yearly_salary",col("yearly_salary").cast("float")).show(5)

+----------+------+----------+---------------+------+-------+-----------------+---------------+-------------+
|First Name|Gender|Start Date|Last Login Time|Salary|Bonus %|Senior Management|           Team|yearly_salary|
+----------+------+----------+---------------+------+-------+-----------------+---------------+-------------+
|   Douglas|  Male|  8/6/1993|       12:42 PM| 97308|  6.945|             true|      Marketing|    1167696.0|
|    Thomas|  Male| 3/31/1996|        6:53 AM| 61933|   4.17|             true|           null|     743196.0|
|     Maria|Female| 4/23/1993|       11:17 AM|130590| 11.858|            false|        Finance|    1567080.0|
|     Jerry|  Male|  3/4/2005|        1:00 PM|138705|   9.34|             true|        Finance|    1664460.0|
|     Larry|  Male| 1/24/1998|        4:47 PM|101004|  1.389|             true|Client Services|    1212048.0|
+----------+------+----------+---------------+------+-------+-----------------+---------------+-------------+
only showi

# withColumnRenamed()
The withColumnRenamed() function

In [32]:
df.withColumnRenamed("First Name","name").show(5)


+-------+------+----------+---------------+------+-------+-----------------+---------------+-------------+
|   name|Gender|Start Date|Last Login Time|Salary|Bonus %|Senior Management|           Team|yearly_salary|
+-------+------+----------+---------------+------+-------+-----------------+---------------+-------------+
|Douglas|  Male|  8/6/1993|       12:42 PM| 97308|  6.945|             true|      Marketing|      1167696|
| Thomas|  Male| 3/31/1996|        6:53 AM| 61933|   4.17|             true|           null|       743196|
|  Maria|Female| 4/23/1993|       11:17 AM|130590| 11.858|            false|        Finance|      1567080|
|  Jerry|  Male|  3/4/2005|        1:00 PM|138705|   9.34|             true|        Finance|      1664460|
|  Larry|  Male| 1/24/1998|        4:47 PM|101004|  1.389|             true|Client Services|      1212048|
+-------+------+----------+---------------+------+-------+-----------------+---------------+-------------+
only showing top 5 rows



# toDF()
The toDF() function returns a new DataFrame with renaming all columns

In [33]:
dataframe_json.toDF("age", "name").show(5)

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



# drop()
The drop() function drops an existing column from DataFrame.


In [34]:
df.drop("yearly_salary").show(5)

+----------+------+----------+---------------+------+-------+-----------------+---------------+
|First Name|Gender|Start Date|Last Login Time|Salary|Bonus %|Senior Management|           Team|
+----------+------+----------+---------------+------+-------+-----------------+---------------+
|   Douglas|  Male|  8/6/1993|       12:42 PM| 97308|  6.945|             true|      Marketing|
|    Thomas|  Male| 3/31/1996|        6:53 AM| 61933|   4.17|             true|           null|
|     Maria|Female| 4/23/1993|       11:17 AM|130590| 11.858|            false|        Finance|
|     Jerry|  Male|  3/4/2005|        1:00 PM|138705|   9.34|             true|        Finance|
|     Larry|  Male| 1/24/1998|        4:47 PM|101004|  1.389|             true|Client Services|
+----------+------+----------+---------------+------+-------+-----------------+---------------+
only showing top 5 rows



# groupBy()
The groupBy() function splits the data into groups based on same values.


In [35]:
df.groupBy("Gender").count().show(5)

+------+-----+
|Gender|count|
+------+-----+
|  null|  145|
|Female|  431|
|  Male|  424|
+------+-----+



# Summary Statistics()

In [36]:
df.summary("count", "min", "25%", "75%", "max").show()

+-------+----------+------+----------+---------------+------+-------+--------------------+-------------+
|summary|First Name|Gender|Start Date|Last Login Time|Salary|Bonus %|                Team|yearly_salary|
+-------+----------+------+----------+---------------+------+-------+--------------------+-------------+
|  count|       933|   855|      1000|           1000|  1000|   1000|                 957|         1000|
|    min|     Aaron|Female|  1/1/2014|       10:00 AM| 35013|  1.015|Business Development|       420156|
|    25%|      null|  null|      null|           null| 62454|  5.368|                null|       749448|
|    75%|      null|  null|      null|           null|118736| 14.837|                null|      1424832|
|    max|    Willie|  Male|  9/9/2006|        9:59 AM|149908| 19.944|               Sales|      1798896|
+-------+----------+------+----------+---------------+------+-------+--------------------+-------------+



# filter()
The filter() function filters the data by asked condition.


In [37]:

df.filter((df.Salary>50 )&(df.Team.startswith("M"))).show(5)

+----------+------+----------+---------------+------+-------+-----------------+---------+-------------+
|First Name|Gender|Start Date|Last Login Time|Salary|Bonus %|Senior Management|     Team|yearly_salary|
+----------+------+----------+---------------+------+-------+-----------------+---------+-------------+
|   Douglas|  Male|  8/6/1993|       12:42 PM| 97308|  6.945|             true|Marketing|      1167696|
|   Matthew|  Male|  9/5/1995|        2:12 AM|100612| 13.645|            false|Marketing|      1207344|
|     Craig|  Male| 2/27/2000|        7:45 AM| 37598|  7.757|             true|Marketing|       451176|
|   Marilyn|Female| 12/7/1980|        3:16 AM| 73524|  5.207|             true|Marketing|       882288|
|      null|Female| 6/12/2007|        5:25 PM| 58112| 19.414|             null|Marketing|       697344|
+----------+------+----------+---------------+------+-------+-----------------+---------+-------------+
only showing top 5 rows



# Regex()
#Select columns by regular expression

In [ ]:
# #
# Single characters:
# •	\d – Identifies numbers between 0 to 9
# •	\w – Identifies all upper and lower case alphabets and numbers from 0 to 9 [A-Z a-z  0 -9]
# •	\s – Whitespace
# •	. – Any character

# We also have set of quantifiers that guide how many characters to search 

# Quantifiers:
# •	* - 0 or more characters
# •	+ - 1 or more characters
# •	? – 0 or 1 characters
# •	{min,max}- specify the range, any characters between the range including min, max
# •	{n} – Exactly n characters


In [38]:
df.select(df.colRegex("`^.*name*`")).show()

+----------+
|First Name|
+----------+
|   Douglas|
|    Thomas|
|     Maria|
|     Jerry|
|     Larry|
|    Dennis|
|      Ruby|
|      null|
|    Angela|
|   Frances|
|    Louise|
|     Julie|
|   Brandon|
|      Gary|
|  Kimberly|
|   Lillian|
|    Jeremy|
|     Shawn|
|     Diana|
|     Donna|
+----------+
only showing top 20 rows



In [39]:
# Select All columns from List
columns = ["First Name","Gender","Salary","Team"]
df.select(*columns).show()

+----------+------+------+--------------------+
|First Name|Gender|Salary|                Team|
+----------+------+------+--------------------+
|   Douglas|  Male| 97308|           Marketing|
|    Thomas|  Male| 61933|                null|
|     Maria|Female|130590|             Finance|
|     Jerry|  Male|138705|             Finance|
|     Larry|  Male|101004|     Client Services|
|    Dennis|  Male|115163|               Legal|
|      Ruby|Female| 65476|             Product|
|      null|Female| 45906|             Finance|
|    Angela|Female| 95570|         Engineering|
|   Frances|Female|139852|Business Development|
|    Louise|Female| 63241|                null|
|     Julie|Female|102508|               Legal|
|   Brandon|  Male|112807|     Human Resources|
|      Gary|  Male|109831|               Sales|
|  Kimberly|Female| 41426|             Finance|
|   Lillian|Female| 59414|             Product|
|    Jeremy|  Male| 90370|     Human Resources|
|     Shawn|  Male|111737|             P

In [40]:
# Select All columns
df.select([col for col in df.columns]).show()
df.select("*").show()

+----------+------+----------+---------------+------+-------+-----------------+--------------------+-------------+
|First Name|Gender|Start Date|Last Login Time|Salary|Bonus %|Senior Management|                Team|yearly_salary|
+----------+------+----------+---------------+------+-------+-----------------+--------------------+-------------+
|   Douglas|  Male|  8/6/1993|       12:42 PM| 97308|  6.945|             true|           Marketing|      1167696|
|    Thomas|  Male| 3/31/1996|        6:53 AM| 61933|   4.17|             true|                null|       743196|
|     Maria|Female| 4/23/1993|       11:17 AM|130590| 11.858|            false|             Finance|      1567080|
|     Jerry|  Male|  3/4/2005|        1:00 PM|138705|   9.34|             true|             Finance|      1664460|
|     Larry|  Male| 1/24/1998|        4:47 PM|101004|  1.389|             true|     Client Services|      1212048|
|    Dennis|  Male| 4/18/1987|        1:35 AM|115163| 10.125|            false| 

# groupBy()
The groupBy() function splits the data into groups based on same values.



In [42]:
df.groupBy("Team").count().show()

+--------------------+-----+
|                Team|count|
+--------------------+-----+
|               Sales|   94|
|         Engineering|   92|
|                null|   43|
|Business Development|  101|
|             Finance|  102|
|     Client Services|  106|
|        Distribution|   90|
|               Legal|   88|
|           Marketing|   98|
|             Product|   95|
|     Human Resources|   91|
+--------------------+-----+



# agg()
The agg() function computes aggregates and returns the result as a columns of a DataFrame with using functions like sum,avg,max,min,count.


In [43]:
from pyspark.sql import functions as F
df.agg(F.min(df.Salary),F.max(df.Salary),F.avg(df.Salary),F.sum(df.Salary)).show()

+-----------+-----------+-----------+-----------+
|min(Salary)|max(Salary)|avg(Salary)|sum(Salary)|
+-----------+-----------+-----------+-----------+
|      35013|     149908|  90662.181|   90662181|
+-----------+-----------+-----------+-----------+



# sort() & orderBy()
The sort() and orderBy() functions sort the data of a column either alphabetic or numeric in ascending or descending order.

In [44]:
df.sort("Salary").show(5)

+----------+------+----------+---------------+------+-------+-----------------+---------------+-------------+
|First Name|Gender|Start Date|Last Login Time|Salary|Bonus %|Senior Management|           Team|yearly_salary|
+----------+------+----------+---------------+------+-------+-----------------+---------------+-------------+
|   Michael|  Male| 7/30/1993|        5:35 PM| 35013| 14.879|            false|        Product|       420156|
|     Kevin|  Male| 3/25/1982|        7:31 AM| 35061|  5.128|            false|          Legal|       420732|
|    Steven|  Male| 3/30/1980|        9:20 PM| 35095|  8.379|             true|Client Services|       421140|
|   Matthew|  Male|  1/2/2013|       10:33 PM| 35203|  18.04|            false|Human Resources|       422436|
|   Cynthia|Female|  7/5/1986|        1:24 AM| 35381| 11.749|            false|        Finance|       424572|
+----------+------+----------+---------------+------+-------+-----------------+---------------+-------------+
only showi

In [45]:
df.orderBy(desc("Salary")).show(5)

+----------+------+----------+---------------+------+-------+-----------------+---------------+-------------+
|First Name|Gender|Start Date|Last Login Time|Salary|Bonus %|Senior Management|           Team|yearly_salary|
+----------+------+----------+---------------+------+-------+-----------------+---------------+-------------+
| Katherine|Female| 8/13/1996|       12:21 AM|149908| 18.912|            false|        Finance|      1798896|
|      Rose|Female| 5/28/2015|        8:40 AM|149903|   5.63|            false|Human Resources|      1798836|
|   Cynthia|Female| 7/12/2006|        8:55 AM|149684|  7.864|            false|        Product|      1796208|
|      null|Female| 2/23/2005|        9:50 PM|149654|  1.825|             null|          Sales|      1795848|
|     Kathy|Female| 3/18/2000|        7:26 PM|149563| 16.991|             true|        Finance|      1794756|
+----------+------+----------+---------------+------+-------+-----------------+---------------+-------------+
only showi

# fillna()
The fillna() function replaces null values with one that we desiganted.



In [46]:
df.fillna({"Senior Management":"true","salary":5000}).show(5)

+----------+------+----------+---------------+------+-------+-----------------+---------------+-------------+
|First Name|Gender|Start Date|Last Login Time|Salary|Bonus %|Senior Management|           Team|yearly_salary|
+----------+------+----------+---------------+------+-------+-----------------+---------------+-------------+
|   Douglas|  Male|  8/6/1993|       12:42 PM| 97308|  6.945|             true|      Marketing|      1167696|
|    Thomas|  Male| 3/31/1996|        6:53 AM| 61933|   4.17|             true|           null|       743196|
|     Maria|Female| 4/23/1993|       11:17 AM|130590| 11.858|            false|        Finance|      1567080|
|     Jerry|  Male|  3/4/2005|        1:00 PM|138705|   9.34|             true|        Finance|      1664460|
|     Larry|  Male| 1/24/1998|        4:47 PM|101004|  1.389|             true|Client Services|      1212048|
+----------+------+----------+---------------+------+-------+-----------------+---------------+-------------+
only showi

# join()
The join() function joins a DataFrame with another DataFrame by matching specified common column with using the chosen join expression.


In [47]:
df2=df
df_join=df.join(df2,"Start Date","inner").show()

+----------+----------+------+---------------+------+-------+-----------------+--------------------+-------------+----------+------+---------------+------+-------+-----------------+--------------------+-------------+
|Start Date|First Name|Gender|Last Login Time|Salary|Bonus %|Senior Management|                Team|yearly_salary|First Name|Gender|Last Login Time|Salary|Bonus %|Senior Management|                Team|yearly_salary|
+----------+----------+------+---------------+------+-------+-----------------+--------------------+-------------+----------+------+---------------+------+-------+-----------------+--------------------+-------------+
|  8/6/1993|   Douglas|  Male|       12:42 PM| 97308|  6.945|             true|           Marketing|      1167696|   Douglas|  Male|       12:42 PM| 97308|  6.945|             true|           Marketing|      1167696|
| 3/31/1996|    Thomas|  Male|        6:53 AM| 61933|   4.17|             true|                null|       743196|     Bobby|  Male|

# union()
The union() function returns a new DataFrame bonding union of rows in current and another DataFrame.


In [1]:

df.union(df2)

NameError: name 'df' is not defined

# alias()
The alias() function renames the new DataFrame column while displaying DataFrame.

In [50]:
df.agg(F.min(df.Salary).alias("min"),F.max(df.Salary).alias("max")).show(5)

+-----+------+
|  min|   max|
+-----+------+
|35013|149908|
+-----+------+



# toJSON()
The toJSON() function converts a DataFrame rows into a JSON documents.


In [51]:
df.toJSON().take(5)

['{"First Name":"Douglas","Gender":"Male","Start Date":"8/6/1993","Last Login Time":"12:42 PM","Salary":97308,"Bonus %":6.945,"Senior Management":true,"Team":"Marketing","yearly_salary":1167696}',
 '{"First Name":"Thomas","Gender":"Male","Start Date":"3/31/1996","Last Login Time":"6:53 AM","Salary":61933,"Bonus %":4.17,"Senior Management":true,"yearly_salary":743196}',
 '{"First Name":"Maria","Gender":"Female","Start Date":"4/23/1993","Last Login Time":"11:17 AM","Salary":130590,"Bonus %":11.858,"Senior Management":false,"Team":"Finance","yearly_salary":1567080}',
 '{"First Name":"Jerry","Gender":"Male","Start Date":"3/4/2005","Last Login Time":"1:00 PM","Salary":138705,"Bonus %":9.34,"Senior Management":true,"Team":"Finance","yearly_salary":1664460}',
 '{"First Name":"Larry","Gender":"Male","Start Date":"1/24/1998","Last Login Time":"4:47 PM","Salary":101004,"Bonus %":1.389,"Senior Management":true,"Team":"Client Services","yearly_salary":1212048}']

# toPandas()
The toPandas() returns the Spark DataFrame into Pandas DataFrame.



In [52]:
df.toPandas().head()

,First Name,Gender,Start Date,Last Login Time,Salary,Bonus %,Senior Management,Team,yearly_salary
0,Douglas,Male,8/6/1993,12:42 PM,97308,6.945,True,Marketing,1167696
1,Thomas,Male,3/31/1996,6:53 AM,61933,4.170,True,None,743196
2,Maria,Female,4/23/1993,11:17 AM,130590,11.858,False,Finance,1567080
3,Jerry,Male,3/4/2005,1:00 PM,138705,9.340,True,Finance,1664460
4,Larry,Male,1/24/1998,4:47 PM,101004,1.389,True,Client Services,1212048


# cache()
The cache() function caches the results into an Spark RDD in memory, so that any re-runs of the cached DataFrame there will be significant performance benefit about processing time.



In [53]:
df.cache()

DataFrame[First Name: string, Gender: string, Start Date: string, Last Login Time: string, Salary: int, Bonus %: double, Senior Management: boolean, Team: string, yearly_salary: int]

In [54]:
df.is_cached

True

# rdd
The rdd function converts Spark DataFrame into Spark RDD.


In [55]:
df.rdd.take(5)

[Row(First Name='Douglas', Gender='Male', Start Date='8/6/1993', Last Login Time='12:42 PM', Salary=97308, Bonus %=6.945, Senior Management=True, Team='Marketing', yearly_salary=1167696),
 Row(First Name='Thomas', Gender='Male', Start Date='3/31/1996', Last Login Time='6:53 AM', Salary=61933, Bonus %=4.17, Senior Management=True, Team=None, yearly_salary=743196),
 Row(First Name='Maria', Gender='Female', Start Date='4/23/1993', Last Login Time='11:17 AM', Salary=130590, Bonus %=11.858, Senior Management=False, Team='Finance', yearly_salary=1567080),
 Row(First Name='Jerry', Gender='Male', Start Date='3/4/2005', Last Login Time='1:00 PM', Salary=138705, Bonus %=9.34, Senior Management=True, Team='Finance', yearly_salary=1664460),
 Row(First Name='Larry', Gender='Male', Start Date='1/24/1998', Last Login Time='4:47 PM', Salary=101004, Bonus %=1.389, Senior Management=True, Team='Client Services', yearly_salary=1212048)]

# columns
The columns function returns all column names in a list format.



In [ ]:
df.columns

# dtypes
The dtypes function returns all column names and their data types in a list format.


In [56]:
df.dtypes

[('First Name', 'string'),
 ('Gender', 'string'),
 ('Start Date', 'string'),
 ('Last Login Time', 'string'),
 ('Salary', 'int'),
 ('Bonus %', 'double'),
 ('Senior Management', 'boolean'),
 ('Team', 'string'),
 ('yearly_salary', 'int')]

The schema function returns the schema of the DataFrame as a StructType.


In [57]:
df.schema

StructType([StructField('First Name', StringType(), True), StructField('Gender', StringType(), True), StructField('Start Date', StringType(), True), StructField('Last Login Time', StringType(), True), StructField('Salary', IntegerType(), True), StructField('Bonus %', DoubleType(), True), StructField('Senior Management', BooleanType(), True), StructField('Team', StringType(), True), StructField('yearly_salary', IntegerType(), True)])

# write & save
The write and save functions saves the DataFrame into a storage in a specified format.


In [58]:
df.write.save("customers.csv",format="csv")

In [59]:
df.select("First Name","Team").write.save("customers_sel_fields.csv",format="csv")

In [60]:
from pyspark.sql.functions import trim,initcap,lower,avg,round


df.withColumn("firstname",trim(initcap("First Name")))\
.withColumn("SeniorManagement",trim("Senior Management"))\
.withColumn("Department",lower(when(df.Team.isNull(),"unknown")\
.otherwise(df.Team)))\
.withColumn("Salary",when(df.Salary.isNull(),df.agg(round(avg("Salary"))).head()[0])\
.otherwise(df.Salary).cast("float")).show(10)

+----------+------+----------+---------------+--------+-------+-----------------+--------------------+-------------+---------+----------------+--------------------+
|First Name|Gender|Start Date|Last Login Time|  Salary|Bonus %|Senior Management|                Team|yearly_salary|firstname|SeniorManagement|          Department|
+----------+------+----------+---------------+--------+-------+-----------------+--------------------+-------------+---------+----------------+--------------------+
|   Douglas|  Male|  8/6/1993|       12:42 PM| 97308.0|  6.945|             true|           Marketing|      1167696|  Douglas|            true|           marketing|
|    Thomas|  Male| 3/31/1996|        6:53 AM| 61933.0|   4.17|             true|                null|       743196|   Thomas|            true|             unknown|
|     Maria|Female| 4/23/1993|       11:17 AM|130590.0| 11.858|            false|             Finance|      1567080|    Maria|           false|             finance|
|     Jerr